In [ ]:
!java -version
!sudo apt-cache search java | awk '{print($1)}' | grep -E -e '^(ia32-)?(sun|oracle)-java' -e '^openjdk-' -e '^icedtea' -e '^(default|gcj)-j(re|dk)' -e '^gcj-(.*)-j(re|dk)' -e 'java-common' | xargs sudo apt-get -y remove
!sudo apt-get -y autoremove
!sudo dpkg -l | grep ^rc | awk '{print($2)}' | xargs sudo apt-get -y purge
!sudo bash -c 'ls -d /home/*/.java' | xargs sudo rm -rf
#!sudo rm -rf /usr/lib/jvm/*
# Install OpenJDK 17
!curl -SsL https://playit-cloud.github.io/ppa/key.gpg | sudo apt-key add -
!sudo curl -SsL -o /etc/apt/sources.list.d/playit-cloud.list https://playit-cloud.github.io/ppa/playit-cloud.list
!sudo apt update &>/dev/null && echo "Cache 成功更新." || echo "Cache更新失败,您可能会受到没有更新的安装包文件."
#!wget -qO - https://adoptopenjdk.jfrog.io/adoptopenjdk/api/gpg/key/public | sudo apt-key add -
#!sudo add-apt-repository --yes https://adoptopenjdk.jfrog.io/adoptopenjdk/deb/ &>/dev/null || echo "Failed to add repo. Still can be ignored if openjdk17 gets installed."
!sudo apt-get install openjdk-17-jre-headless
print("安装成功!")
!java -version
java_ver = !java -version 2>&1 | awk -F[\"\.] -v OFS=. 'NR==1{print $2}'
if java_ver[0] == "17" :
  print("Openjdk17运行正常.")
else:
  print("Openjdk17似乎没有被安装,或者没有正常工作.返回一个Java版本", java_ver[0], ". 可能会降低运行的性能 Minecraft 1.17 或者更高版本的 Minecraft 可能会运行失败.")

In [ ]:
import ipywidgets as widgets
from IPython.display import display
api = "https://api.github.com/repos/lemonfan-maker/Minecraft-Java-On-Action/releases/latest"
import requests
response = requests.get(api)
print("是否创建新的世界?创建新的世界会覆盖原有存档.(输入完成后回车)")
text = widgets.Text()
text.on_submit(print_value)
if text == "y":
    print("创建新的世界.")
    import requests
    import json
    version = '1.19.2'
    server_type = 'paper'
    !mkdir "/home/runner/work/Minecraft-Java-On-Action/Minecraft-Java-On-Action/Minecraft-Server/"
    %cd "/home/runner/work/Minecraft-Java-On-Action/Minecraft-Java-On-Action/Minecraft-Server/"
    # Internal init...
    a = requests.get("https://papermc.io/api/v2/projects/paper/versions/" + version)
    #print(a.json()["builds"][-1])
    b = requests.get("https://papermc.io/api/v2/projects/paper/versions/" + version + "/builds/" + str(a.json()["builds"][-1]))
    #print(b.json()["downloads"]["application"]["name"])
    print("https://papermc.io/api/v2/projects/paper/versions/" + version + "/builds/" + str(a.json()["builds"][-1]) + "/downloads/" + b.json()["downloads"]["application"]["name"])
    paperURL = "https://papermc.io/api/v2/projects/paper/versions/" + version + "/builds/" + str(a.json()["builds"][-1]) + "/downloads/" + b.json()["downloads"]["application"]["name"]
    jar_name = {'paper': 'server.jar', 'fabric': 'fabric-installer.jar'}
    url = {
        'paper': (paperURL),
        'fabric': 'https://maven.fabricmc.net/net/fabricmc/fabric-installer/0.7.4/fabric-installer-0.7.4.jar'
        }
    print('下载中...')
    r = requests.get(url[server_type])
    if r.status_code == 200:
        with open('/home/runner/work/Minecraft-Java-On-Action/Minecraft-Java-On-Action/Minecraft-Server/' + jar_name[server_type], 'wb') as f:
            f.write(r.content)
    else:
        print('Error '+ str(r.status_code) + '! 大多数情况下,您选择了一个未知的版本,请检查版本号.如果您认为不应该发生这种情况,请尝试再次运行代码.')
    # Running specific install path.
    if server_type == 'fabric':
        !java -jar fabric-installer.jar server -mcversion $version -downloadMinecraft
    # Saving config
    colabconfig = {"server_type": server_type}
    json.dump(colabconfig, open("githubconfig.json",'w'))
    import os
    %cd "/home/runner/work/Minecraft-Java-On-Action/Minecraft-Java-On-Action/Minecraft-Server/"
    if os.path.isfile("githubconfig.json"):
        colabconfig = json.load(open("githubconfig.json"))
    else:
        colabconfig = {"server_type": "generic"} # using default, if config doesn't exists.
        json.dump(colabconfig, open("githubconfig.json",'w'))
    # Server jar names.
    jar_list = {'paper': 'server.jar', 'fabric': 'fabric-server-launch.jar', 'generic': 'server.jar'}
    jar_name = jar_list[colabconfig["server_type"]]
    # Java arguments.
    if colabconfig["server_type"] == "paper":
        server_flags = "-XX:+UseG1GC -XX:+ParallelRefProcEnabled -XX:MaxGCPauseMillis=200 -XX:+UnlockExperimentalVMOptions -XX:+DisableExplicitGC -XX:+AlwaysPreTouch -XX:G1NewSizePercent=30 -XX:G1MaxNewSizePercent=40 -XX:G1HeapRegionSize=8M -XX:G1ReservePercent=20 -XX:G1HeapWastePercent=5 -XX:G1MixedGCCountTarget=4 -XX:InitiatingHeapOccupancyPercent=15 -XX:G1MixedGCLiveThresholdPercent=90 -XX:G1RSetUpdatingPauseTimePercent=5 -XX:SurvivorRatio=32 -XX:+PerfDisableSharedMem -XX:MaxTenuringThreshold=1 -Dusing.aikars.flags=https://mcflags.emc.gs -Daikars.new.flags=true"
    else:
        server_flags = "" # aiker's flags might negatively impact performance on non-paper servers.
    memory_allocation = "-Xmx5000M -Xms5000M"
    print('Starting server...')
    !java $memory_allocation $server_flags -jar $jar_name nogui
    print('完成!')
    %cd "/home/runner/work/Minecraft-Java-On-Action/Minecraft-Java-On-Action/Minecraft-Server/"
    !sed -i '$d' eula.txt
    !echo "eula=true" >> eula.txt
    import re
    !sudo bash -c "echo 3 > /proc/sys/vm/drop_caches" 
    !sudo rm -rf /etc/playit/playit.toml
    %cd "/home/runner/work/Minecraft-Java-On-Action/Minecraft-Java-On-Action/Minecraft-Server/"
    !rm -rf /home/runner/work/Minecraft-Java-On-Action/Minecraft-Java-On-Action/Minecraft-Server/server.properties
    !cp /home/runner/work/Minecraft-Java-On-Action/Minecraft-Java-On-Action/server.properties /home/runner/work/Minecraft-Java-On-Action/Minecraft-Java-On-Action/Minecraft-Server/
    if os.path.isfile("githubconfig.json"):
        colabconfig = json.load(open("githubconfig.json"))
    else:
        colabconfig = {"server_type": "generic"} # using default, if config doesn't exists.
        json.dump(colabconfig, open("githubconfig.json",'w'))
    # Server jar names.
    jar_list = {'paper': 'server.jar', 'fabric': 'fabric-server-launch.jar', 'generic': 'server.jar'}
    jar_name = jar_list[colabconfig["server_type"]]
    # Java arguments.
    if colabconfig["server_type"] == "paper":
        server_flags = "-XX:+UseG1GC -XX:+ParallelRefProcEnabled -XX:MaxGCPauseMillis=200 -XX:+UnlockExperimentalVMOptions -XX:+DisableExplicitGC -XX:+AlwaysPreTouch -XX:G1NewSizePercent=30 -XX:G1MaxNewSizePercent=40 -XX:G1HeapRegionSize=8M -XX:G1ReservePercent=20 -XX:G1HeapWastePercent=5 -XX:G1MixedGCCountTarget=4 -XX:InitiatingHeapOccupancyPercent=15 -XX:G1MixedGCLiveThresholdPercent=90 -XX:G1RSetUpdatingPauseTimePercent=5 -XX:SurvivorRatio=32 -XX:+PerfDisableSharedMem -XX:MaxTenuringThreshold=1 -Dusing.aikars.flags=https://mcflags.emc.gs -Daikars.new.flags=true"
    else:
        server_flags = "playit" # aiker's flags might negatively impact performance on non-paper servers.
    memory_allocation = "-Xmx5000M -Xms5000M"
    # Chose the tunnle service you want to use
    # Available options: ngrok, argo, playit
    print("可用的通道:ngrok,argo,playit")
    tunnel_service = widgets.Text()
    tunnel_service.on_submit(print_value)
    print("Procedding to use", tunnel_service)
    if tunnel_service == "ngrok":
        !pip -q install pyngrok
        from pyngrok import conf, ngrok
        # Ask for the ngrok authtoken
        print("Get your authtoken from https://dashboard.ngrok.com/auth")
        import getpass
        authtoken = getpass.getpass() # input your Ngrok auth token everytime you run the cell or simply replace "2GJ3VTZPLvaiDzLvxzNO7ED53DE_3mSkpo59sq3f33ciqHLdP" with your token in "double quotes"
        !ngrok authtoken $authtoken # login to ngrok
        # Sets default ngrok region
        conf.get_default().region = 'us'  # Change this to whichever region you want
        # Connect to ngrok
        url = ngrok.connect(25565, 'tcp')
        print('Your server address is ' + ((str(url).split('"')[1::2])[0]).replace('tcp://', ''))
        print('Starting server...')
        !java $memory_allocation $server_flags -jar $jar_name nogui
    elif tunnel_service == "playit":
        !sudo apt install playit
        print('Starting server...')
        !playit & java $memory_allocation $server_flags -jar $jar_name nogui
    else:
        # Download & make argo executable
        !wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
        !chmod +x cloudflared-linux-amd64
        print('Starting server...')
        !./cloudflared-linux-amd64 tunnel --url tcp://127.0.0.1:25565 & java $memory_allocation $server_flags -jar $jar_name nogui

if text == "n":
    print("正在寻找原有存档.")
    if "message" in response.json():
        print("不存在数据包,请重新执行本单元格,在输入框中输入 'n' 并回车,创建新的世界.")
    else:
        print("下载数据包中...")
        !aria2c -x 5 https://github.com/LemonFan-maker/Download-Baidu-On-Action/releases/latest/Minecraft-Server.tgz
        !tar -xvf Minecraft-Server.tgz
        import os
        import re
        import json
        !sudo bash -c "echo 3 > /proc/sys/vm/drop_caches" 
        !sudo rm -rf /etc/playit/playit.toml
        %cd "/home/runner/work/Minecraft-Java-On-Action/Minecraft-Java-On-Action/Minecraft-Server/"
        !cp /home/runner/work/Minecraft-Java-On-Action/Minecraft-Java-On-Action/server.properties /home/runner/work/Minecraft-Java-On-Action/Minecraft-Java-On-Action/Minecraft-Server/
        if os.path.isfile("githubconfig.json"):
            colabconfig = json.load(open("githubconfig.json"))
        else:
            colabconfig = {"server_type": "generic"} # using default, if config doesn't exists.
            json.dump(colabconfig, open("githubconfig.json",'w'))
        # Server jar names.
        jar_list = {'paper': 'server.jar', 'fabric': 'fabric-server-launch.jar', 'generic': 'server.jar'}
        jar_name = jar_list[colabconfig["server_type"]]
    # Java arguments.
        if colabconfig["server_type"] == "paper":
            server_flags = "-XX:+UseG1GC -XX:+ParallelRefProcEnabled -XX:MaxGCPauseMillis=200 -XX:+UnlockExperimentalVMOptions -XX:+DisableExplicitGC -XX:+AlwaysPreTouch -XX:G1NewSizePercent=30 -XX:G1MaxNewSizePercent=40 -XX:G1HeapRegionSize=8M -XX:G1ReservePercent=20 -XX:G1HeapWastePercent=5 -XX:G1MixedGCCountTarget=4 -XX:InitiatingHeapOccupancyPercent=15 -XX:G1MixedGCLiveThresholdPercent=90 -XX:G1RSetUpdatingPauseTimePercent=5 -XX:SurvivorRatio=32 -XX:+PerfDisableSharedMem -XX:MaxTenuringThreshold=1 -Dusing.aikars.flags=https://mcflags.emc.gs -Daikars.new.flags=true"
        else:
            server_flags = "" # aiker's flags might negatively impact performance on non-paper servers.
        memory_allocation = "-Xmx5000M -Xms5000M"
        # Chose the tunnle service you want to use
        # Available options: ngrok, argo, playit
        print("可用的通道:ngrok,argo,playit")
        tunnel_service = widgets.Text()
        tunnel_service.on_submit(print_value)
        print("Procedding to use", tunnel_service)
        if tunnel_service == "ngrok":
            !pip -q install pyngrok
            from pyngrok import conf, ngrok
            # Ask for the ngrok authtoken
            print("Get your authtoken from https://dashboard.ngrok.com/auth")
            import getpass
            authtoken = "2GJ3VTZPLvaiDzLvxzNO7ED53DE_3mSkpo59sq3f33ciqHLdP" # input your Ngrok auth token everytime you run the cell or simply replace "getpass.getpass()" with your token in "double quotes"
            !ngrok authtoken $authtoken # login to ngrok
            # Sets default ngrok region
            conf.get_default().region = 'us'  # Change this to whichever region you want
            # Connect to ngrok
            url = ngrok.connect(25565, 'tcp')
            print('Your server address is ' + ((str(url).split('"')[1::2])[0]).replace('tcp://', ''))
            print('Starting server...')
            !java $memory_allocation $server_flags -jar $jar_name nogui
        elif tunnel_service == "playit":
            !sudo apt install playit
            print('Starting server...')
            !playit & java $memory_allocation $server_flags -jar $jar_name nogui
        else:
            # Download & make argo executable
            !wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
            !chmod +x cloudflared-linux-amd64
            print('Starting server...')
            !./cloudflared-linux-amd64 tunnel --url tcp://127.0.0.1:25565 & java $memory_allocation $server_flags -jar $jar_name nogui